In [1]:
from copy import *
from math import *

def my_format(x):
    if type(x)==int: return str(x)
    if abs(x)<1e-7: return '0'
    try:
        k = int(log10(x))+1
    except Exception as e:
        raise Exception(x,e)
    #print(k,max(4-k,0))
    return ('{:.' + str(max(4-k,0)) + 'f}').format(x)

In [2]:
my_format(12345.6)

'12346'

In [3]:
my_format(1234.56)

'1235'

In [4]:
my_format(123.456)

'123.5'

In [5]:
my_format(12.3456)

'12.35'

In [6]:
my_format(1.23456)

'1.235'

In [7]:
CONVERSION_FACTOR=9.81
class engine:
    def __init__(self,m,thrust_min,thrust_max,isp_min,isp_max,m_f=0):
        self.m_c=m
        self.m_f=m_f
        self.thrust_min=thrust_min
        self.thrust_max=thrust_max
        self.isp_min=isp_min
        self.isp_max=isp_max
        if thrust_min>0:
            assert (thrust_max/isp_max - thrust_min/isp_min)/thrust_min/isp_min < 1e-4,\
                (thrust_max/isp_max - thrust_min/isp_min)/thrust_min/isp_min
        self.consumption = thrust_max/isp_max/CONVERSION_FACTOR
    def __mul__(self, mul):
        return engine(
            self.m_c*mul,
            self.thrust_min*mul,
            self.thrust_max*mul,
            self.isp_min,
            self.isp_max,
            self.m_f*mul
        )
    def __rmul__(self, other):
        return self.__mul__(other)
    def thrust_lim(self,lim):
        return engine(
            self.m_c,
            self.thrust_min*lim,
            self.thrust_max*lim,
            self.isp_min,
            self.isp_max,
            self.m_f
        )
    def __add__(self,other):
        a1=self.consumption/(self.consumption+other.consumption)
        a2=other.consumption/(self.consumption+other.consumption)
        return engine(
            self.m_c+other.m_c,
            self.thrust_min+other.thrust_min,
            self.thrust_max+other.thrust_max,
            a1*self.isp_min+a2*other.isp_min,
            a1*self.isp_max+a2*other.isp_max,
            (self.m_f+other.m_f)
        )
    
    def __repr__(self):
        return 'make_engine('+str(self.m_c/1000)+', '+\
            my_format(self.thrust_min/1000)+', '+\
            my_format(self.thrust_max/1000)+', '+\
            my_format(self.isp_min)+', '+\
            my_format(self.isp_max)+', '+\
            my_format(self.m_f/1000)+\
        ') # '+my_format(self.consumption)

def make_engine(m,thrust_min,thrust_max,isp_min,isp_max,m_f=0):
    return engine(m*1000,thrust_min*1000,thrust_max*1000,isp_min,isp_max,m_f*1000)
    
муравей = make_engine(	0.020	,0.5	,2	    ,80	    ,315	        )
терьер = make_engine(	0.500	,14.783	,60	    ,85	    ,345	        )
гепард = make_engine(	1   	,25	    ,125	,150	,355	        )
пудель = make_engine(	1.750	,64	    ,250	,90	    ,350	        )
волкодав = make_engine(	3.300	,69	    ,375	,70	    ,380	        )
юнец = make_engine(		0.018	,29	    ,32	    ,280	,310	        )
паук = make_engine(		0.020	,1.8	,2	    ,260	,290	        )
свеча = make_engine(	0.080	,15	    ,16	    ,275	,290	        )
стукач = make_engine(	0.900	,108	,120	,275	,305	        )
искра = make_engine(	0.130	,16	    ,20	    ,265	,320	        )
дротик = make_engine(	1.000	,153	,180	,290	,340	        )
РАПИРА = make_engine(	2.000	,162	,180	,275	,305	        )
медведь = make_engine(	1.250	,247	,260	,285	,300	        )
факел = make_engine(	1.250	,205	,240	,265	,310	        )
вертлявый = make_engine(1.500	,167	,215	,250	,320	        )
скиф = make_engine(		1.600	,240	,300	,265	,330	        )
рысь = make_engine(		2.000	,374	,400	,290	,310	        )
шкипер = make_engine(	3.000	,568	,650	,280	,320	        )
вектор = make_engine(	4.000	,936	,1000	,295	,315	        )
мастодонт = make_engine(5.000	,1283	,1350	,290	,305	        )
грохот = make_engine(	6.000	,1379	,1500	,285	,310	        )
носорог = make_engine(	9.000	,1205	,2000	,205	,340	        )
мамонт = make_engine(	15.000	,3746	,4000	,295	,315	        )
курьер = make_engine(	10.000	,1866	,2000	,280	,300	,160    )

блоха = make_engine(	0.450	,163	,192	,140	,165	,1.05	)
молот = make_engine(	0.7585	,198	,227	,170	,195	,2.81	)
кувалда = make_engine(	1.500	,250	,300	,175	,210	,6.15	)
булава = make_engine(	4.500	,594	,670	,195	,220	,19.5	)

null_engine = make_engine(0,0,0,1,1)

class stage:
    def __init__(self,m_c,m_f,engines,aspargarus=0):#atm...
        self.m_c=m_c+engines.m_c
        self.m_f=m_f+engines.m_f
        self.engines = copy(engines)#atm...
        self.engines.m_c=0
        self.engines.m_f=0
        self.aspargarus = aspargarus # показывает, на сколько ступеней дальше передается топливо
    def __mul__(self,mul):
        return stage(mul*self.m_c,
                mul*self.m_f,
                mul*self.engines,
                self.aspargarus
            )
    def __rmul__(self, other):
        return self.__mul__(other)
    def __repr__(self):
        return 'stage('+\
            my_format((self.m_c))+', '+\
            my_format(self.m_f)+', '+\
            repr(self.engines)+\
        ')'
    
def st(m_c,m_f,engines,aspargarus=0):
    return stage((m_c+m_f/8)*1000,m_f*1000,engines,aspargarus)

In [8]:
st(0,0,блоха)

stage(450.0, 1050, make_engine(0.0, 163.0, 192.0, 140, 165, 0) # 118.6)

In [9]:
2*блоха

make_engine(0.9, 326.0, 384.0, 140, 165, 2.100) # 237.2

In [10]:
2*st(0,0,блоха)

stage(900.0, 2100, make_engine(0.0, 326.0, 384.0, 140, 165, 0) # 237.2)

In [11]:
def build_stage(stages):
    # stages - список списков ступеней
    lst = stages[-1] # last stage
    if len(lst)==1:
        del stages[-1]
        return lst[0]
    # выделяем отдельные asp_list-ы
    asp_lists = [] # aspargarus lists - список из [asp_list,asp_engines,asp_time]
    # asp_lists список из asp_meta_list
    #   asp_meta_list[0] - asp_list
    #     asp_list - список индексов в lst
    #   asp_meta_list[1] - asp_engine
    #   asp_meta_list[2] - asp_time
    not_used = [i for i in range(len(lst))]
    while len(not_used)>0:
        cur = not_used[-1]
        del not_used[-1]
        asp_list=[cur]
        asp_engines = lst[cur].engines
        while lst[cur].aspargarus:
            cur-=lst[cur].aspargarus
            assert cur in not_used, 'неправильная последовательность передачи топлива'
            not_used.remove(cur)
            asp_list.insert(0,cur)
            asp_engines += lst[cur].engines
        asp_lists.insert(0,[asp_list,asp_engines])
    # вычисляем время работы 1 ступени для каждого asp_list-а
    for i in range(len(asp_lists)):
        asp_lists[i].append(lst[asp_lists[i][0][-1]].m_f/asp_lists[i][1].consumption)
    # проверяем, чтобы у крайней ступени было минимальное время
    last_time = asp_lists[-1][2]
    for asp_meta_list in asp_lists[:-1]:
        assert asp_meta_list[2]>last_time, 'ступень '+\
            str(len(stages)-1)+'-'+str(asp_meta_list[0][-1])+' работает слижком быстро'
    # формируем ступень и остаток
    tot_constr = lst[asp_lists[-1][0][-1]].m_c
    tot_engines = copy(null_engine)
    tot_fuel = 0
    for asp_meta_list in asp_lists:
        tot_engines+=asp_meta_list[1]
        asp_fuel = asp_meta_list[1].consumption*last_time
        tot_fuel+=asp_fuel
        lst[asp_meta_list[0][-1]].m_f-=asp_fuel
    assert lst[asp_lists[-1][0][-1]].m_f<1
    del lst[-1]
    return stage(tot_constr,tot_fuel,tot_engines)

def build(*stages):
    stages = list(stages)
    for i in range(len(stages)):
        if type(stages[i])!=list:
            stages[i] = [stages[i]]
            
    building = []
    while len(stages):
        building.insert(0,build_stage(stages))
    return building

In [12]:
build(st(0,0,блоха))

[stage(450.0, 1050, make_engine(0.0, 163.0, 192.0, 140, 165, 0) # 118.6)]

In [13]:
build(2*st(0,0,блоха),st(0,0,булава))

[stage(900.0, 2100, make_engine(0.0, 326.0, 384.0, 140, 165, 0) # 237.2),
 stage(4500, 19500, make_engine(0.0, 594.0, 670.0, 195, 220, 0) # 310.4)]

In [14]:
build([st(0,0,булава),2*st(0,0,блоха)])

[stage(4500, 16752, make_engine(0.0, 594.0, 670.0, 195, 220, 0) # 310.4),
 stage(900.0, 4848, make_engine(0.0, 920.0, 1054, 171.2, 196.2, 0) # 547.7)]

In [15]:
2*st(0,0,блоха)

stage(900.0, 2100, make_engine(0.0, 326.0, 384.0, 140, 165, 0) # 237.2)

In [16]:
st(0,0,булава)

stage(4500, 19500, make_engine(0.0, 594.0, 670.0, 195, 220, 0) # 310.4)

In [17]:
from IPython.core.display import HTML
global_g = 9.81

class satge_info:
    def __init__(self,m_i,m_f,dv,v_f,w_t,twr_i,twr_m,twr_f):
        self.m_i   = m_i   # mass initial
        self.m_f   = m_f   # mass final
        self.dv    = dv    # delta v
        self.v_f   = v_f   # v final
        self.w_t   = w_t   # work_time
        self.twr_i = twr_i # thrust to weight ratio initial
        self.twr_m = twr_m # thrust to weight ratio mean
        self.twr_f = twr_f # thrust to weight ratio final
    def __repr__(self):
        return 'stage_info('+\
            my_format(self.m_i)+', '+\
            my_format(self.m_f)+', '+\
            my_format(self.dv)+', '+\
            my_format(self.v_f)+', '+\
            my_format(self.w_t)+', '+\
            my_format(self.twr_i)+', '+\
            my_format(self.twr_m)+', '+\
            my_format(self.twr_f)+', '+\
        ')'
            #'m_i='+\  'dv='+  'twr_m='+  'twr_f='+
    def html_row(self):
        return '<tr>'+\
            '<td>'+my_format(self.m_i)+'</td>'+\
            '<td>'+my_format(self.m_f)+'</td>'+\
            '<td>'+my_format(self.dv)+'</td>'+\
            '<td>'+my_format(self.v_f)+'</td>'+\
            '<td>'+my_format(self.w_t)+'</td>'+\
            '<td>'+my_format(self.twr_i)+'</td>'+\
            '<td>'+my_format(self.twr_m)+'</td>'+\
            '<td>'+my_format(self.twr_f)+'</td>'+\
        '</tr>'
    @staticmethod
    def html_head():
        return '<tr>'+\
            '<td>m_i,т</td>'+\
            '<td>m_f,т</td>'+\
            '<td>dv,м/с   </td>'+\
            '<td>v_f,м/с  </td>'+\
            '<td>w_t,с  </td>'+\
            '<td>twr_i,a/g</td>'+\
            '<td>twr_m,a/g</td>'+\
            '<td>twr_f,a/g</td>'+\
        '</tr>'
        
        

def calc(*stages):
    if len(stages)==1 and type(stages[0])==list:
        stages = stages[0]
    stages = list(stages)
    
    m_f_l = [] # mass final list
    m_i_l = [] # mass initial list
    cur_m = 0
    for x in stages:
        cur_m+=x.m_c
        m_f_l.append(cur_m)
        cur_m+=x.m_f
        m_i_l.append(cur_m)
    info = []
    cur_v = 0
    html_s = ''
    for i in reversed(range(len(stages))):
        en = stages[i].engines
        m_i = m_i_l[i]
        m_f = m_f_l[i]
        dv = en.isp_max*log(m_i/m_f)*CONVERSION_FACTOR
        cur_v+=dv
        v_f = cur_v
        w_t = (m_i-m_f)/en.consumption
        twr_i = en.thrust_max/m_i/global_g
        twr_f = en.thrust_max/m_f/global_g
        twr_m = dv/w_t/global_g
        stage = satge_info(m_i/1000,m_f/1000,dv,v_f,w_t,twr_i,twr_m,twr_f)
        html_s = stage.html_row()+html_s
        info.insert(0,stage)
    display(HTML('<table>'+stage.html_head()+html_s+'</table>'))
#    return info

In [18]:
calc(st(0,0,блоха))
#     m_i,        m_f,    dv,     v_f,    w_t,    twr_i,  twr_m,  twr_f

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
1.500,0.450,1949,1949,8.852,13.05,22.44,43.49


In [19]:
calc(st(0,0,булава),5*st(0.01,0,блоха))
#     m_i,        m_f,    dv,     v_f,    w_t,    twr_i,  twr_m,  twr_f

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
24.00,4.500,3613,3907,62.81,2.846,5.863,15.18
31.55,26.30,294.6,294.6,8.852,3.102,3.393,3.721


In [20]:
calc(st(0,2,терьер))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
2.750,0.750,4397,4397,112.8,2.224,3.973,8.155


In [21]:
calc(st(0,2,вертлявый))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
3.750,1.750,2393,2393,29.20,5.844,8.352,12.52


In [22]:
calc(st(1.086,2,терьер),st(0.44,6,вертлявый),2*st(0.05,0,молот),2*st(0.05,0,молот))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
3.836,1.836,2494,5627,112.8,1.594,2.253,3.331
12.53,6.526,2047,3133,87.61,1.750,2.382,3.358
19.76,14.14,640.1,1087,23.68,2.342,2.755,3.272
27.00,21.38,446.4,446.4,23.68,1.714,1.922,2.165


In [23]:
calc(st(1.086,2,терьер),st(0.44,6,вертлявый),4*st(0.05,0,молот.thrust_lim(0.6)))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
3.836,1.836,2494,5570,112.8,1.594,2.253,3.331
12.53,6.526,2047,3077,87.61,1.750,2.382,3.358
27.00,15.76,1030,1030,39.47,2.057,2.660,3.524


In [24]:
calc(build(st(1.086,2,терьер),[st(0.44,6,вертлявый),4*st(0.05,0,молот.thrust_lim(1))]))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
3.836,1.836,2494,5443,112.8,1.594,2.253,3.331
10.90,6.526,1612,2949,63.93,2.010,2.570,3.358
27.00,14.14,1338,1338,23.68,4.240,5.758,8.097


In [25]:
calc(build(st(1.086,2,терьер),[st(0.44,6,вертлявый),4*st(0.05,0,молот.thrust_lim(0.6))]))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
3.836,1.836,2494,5340,112.8,1.594,2.253,3.331
9.823,6.526,1284,2846,48.14,2.231,2.718,3.358
27.00,13.06,1562,1562,39.47,2.869,4.036,5.932


In [26]:
#zonder
calc(st(0.135,0.2,муравей),st(0.446,5,вертлявый))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
0.380,0.180,2309,5420,309.0,0.537,0.762,1.133
7.951,2.951,3111,3111,73.00,2.756,4.345,7.427


In [27]:
# kerbal-x
calc(build(st(4.5,8,пудель),
[st(0.5,48,грохот),
     2*st(0.1,6,вертлявый,1),
     2*st(0.1,6,вертлявый,1),
     2*st(0.1,6,вертлявый,1)
]))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
15.25,7.250,2553,6700,109.9,1.671,2.369,3.515
75.75,27.75,3054,4147,97.32,2.019,3.199,5.510
92.45,80.45,425.8,1093,19.04,2.128,2.279,2.445
109.2,97.15,358.3,667.5,15.64,2.204,2.335,2.476
125.8,113.8,309.2,309.2,13.27,2.260,2.375,2.498


In [28]:
# kerbal-x
calc(build(st(4.5,8,пудель),
[st(0.5,48,носорог),
     2*st(0.1,6,вертлявый,1),
     2*st(0.1,6,вертлявый,1),
     2*st(0.1,6,вертлявый,1)
]))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
15.25,7.250,2553,6822,109.9,1.671,2.369,3.515
78.75,30.75,3137,4269,80.05,2.589,3.994,6.630
95.45,83.45,443.2,1132,16.29,2.595,2.773,2.968
112.2,100.2,370.5,688.8,13.74,2.600,2.749,2.911
128.8,116.8,318.3,318.3,11.87,2.603,2.732,2.870


In [29]:
# odin
calc(build(st(0.98,2,вертлявый),[st(0.08,6,вертлявый),st(0.055,0,молот.thrust_lim(0.51))]))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
4.730,2.730,1725,3921,29.20,4.633,6.023,8.028
9.880,7.060,1055,2195,41.17,2.218,2.612,3.104
16.68,10.69,1140,1140,46.43,2.021,2.504,3.153


In [30]:
# odin
calc(build(st(0.98,3,вертлявый),[st(0.08,5,вертлявый),st(0.055,0,молот.thrust_lim(0.51))]))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
5.855,2.855,2255,4034,43.80,3.743,5.247,7.677
9.880,8.060,639.1,1780,26.57,2.218,2.452,2.719
16.68,10.69,1140,1140,46.43,2.021,2.504,3.153


In [31]:
# odin
calc(build(st(0.98,4,вертлявый),[st(0.08,4,вертлявый),st(0.055,0,молот.thrust_lim(0.51))]))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
6.980,2.980,2672,4084,58.40,3.140,4.663,7.355
9.880,9.060,272.0,1412,11.97,2.218,2.316,2.419
16.68,10.69,1140,1140,46.43,2.021,2.504,3.153


In [32]:
# odin
calc(build([st(1,8,вертлявый),st(0.055,0,молот.thrust_lim(0.51))]))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
8.320,3.500,2718,4011,70.38,2.634,3.937,6.262
15.12,9.133,1293,1293,46.43,2.229,2.839,3.692


In [33]:
calc(build(st(64,128,6*вектор),st(1,256+128+8*10,(8+4)*вектор),st(1,1000,16*вектор)))

"m_i,т","m_f,т","dv,м/с","v_f,м/с","w_t,с","twr_i,a/g","twr_m,a/g","twr_f,a/g"
232.0,104.0,2479,7297,65.92,2.636,3.834,5.881
803.0,339.0,2665,4818,119.5,1.523,2.273,3.608
1993,993.0,2153,2153,193.1,0.818,1.136,1.642


In [35]:
NoneType = type(None)
builtin_function_or_method = type(getattr)
function = type(lambda x:x)
module = type(copy)

In [36]:
%%javascript
window.executePython = function(python) {
    return new Promise((resolve, reject) => {
        var callbacks = {
            iopub: {
                output: (data) => resolve(data.content.text.trim())
            }
        };
        Jupyter.notebook.kernel.execute(`print(${python})`, callbacks);    
    });
}

<IPython.core.display.Javascript object>

In [37]:
from IPython.core.display import HTML

input_form = """
<div style="background-color:gainsboro; border:solid black; width:300px; padding:20px;">
Variable Name: <input type="text" id="var_name" value="foo"><br>
Variable Value: <input type="text" id="var_value" value="bar"><br>
<button onclick="set_value()">Set Value</button>
</div>
"""

javascript = """
<script type="text/Javascript">
    function set_value(){
        var var_name = document.getElementById('var_name').value;
        var var_value = document.getElementById('var_value').value;
        var command = var_name + " = '" + var_value + "'";
        console.log("Executing Command: " + command);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command);
    }
</script>
"""

display(HTML(input_form + javascript))

In [39]:
foo

'bar'

In [44]:
import numpy as np
help(np.linalg)

Help on package numpy.linalg in numpy:

NAME
    numpy.linalg

DESCRIPTION
    Core Linear Algebra Tools
    -------------------------
    Linear algebra basics:
    
    - norm            Vector or matrix norm
    - inv             Inverse of a square matrix
    - solve           Solve a linear system of equations
    - det             Determinant of a square matrix
    - lstsq           Solve linear least-squares problem
    - pinv            Pseudo-inverse (Moore-Penrose) calculated using a singular
                      value decomposition
    - matrix_power    Integer power of a square matrix
    
    Eigenvalues and decompositions:
    
    - eig             Eigenvalues and vectors of a square matrix
    - eigh            Eigenvalues and eigenvectors of a Hermitian matrix
    - eigvals         Eigenvalues of a square matrix
    - eigvalsh        Eigenvalues of a Hermitian matrix
    - qr              QR decomposition of a matrix
    - svd             Singular value decomposition 

In [46]:
from Cython.Compiler.Version import version as cython_version

In [52]:
import os
os.chdir(r'C:\Users\feelus\Repos\__foreign\numpy')
!dir

 ’®¬ ў гбва®©бвўҐ C Ё¬ҐҐв ¬ҐвЄг sys
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 581B-FAA0

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Users\feelus\Repos\__foreign\numpy

01.04.2020  20:53    <DIR>          .
01.04.2020  20:53    <DIR>          ..
01.04.2020  20:34    <DIR>          .circleci
01.04.2020  20:34               235 .codecov.yml
01.04.2020  20:34                75 .coveragerc
01.04.2020  20:34                19 .ctags.d
01.04.2020  20:34    <DIR>          .dependabot
01.04.2020  21:10    <DIR>          .git
01.04.2020  20:34               133 .gitattributes
01.04.2020  20:34    <DIR>          .github
01.04.2020  20:34             3я998 .gitignore
01.04.2020  20:34               218 .gitmodules
01.04.2020  21:13    <DIR>          .idea
01.04.2020  20:34               573 .lgtm.yml
01.04.2020  20:34            18я631 .mailmap
01.04.2020  20:34             3я188 .travis.yml
01.04.2020  20:34             9я677 azure-pipelines.yml
01.04.2020  20:34             2я576 azure-steps-windows.yml
01.04.2020  20:34    <DIR>       

In [54]:
!python setup.py build

2223
numpy/random\_bounded_integers.pxd.in has not changed
Processing numpy/random\bit_generator.pyx
Processing numpy/random\mtrand.pyx
Processing numpy/random\_bounded_integers.pyx.in
Processing numpy/random\_common.pyx
Processing numpy/random\_generator.pyx
Processing numpy/random\_mt19937.pyx
Processing numpy/random\_pcg64.pyx
Processing numpy/random\_philox.pyx
Processing numpy/random\_sfc64.pyx
Cythonizing sources
blas_opt_info:
blas_mkl_info:
No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
customize MSVCCompiler
  libraries mkl_rt not found in ['C:\\ProgramData\\Miniconda3\\lib', 'C:\\', 'C:\\ProgramData\\Miniconda3\\libs']
  NOT AVAILABLE

blis_info:
  libraries blis not found in ['C:\\ProgramData\\Miniconda3\\lib', 'C:\\', 'C:\\ProgramData\\Miniconda3\\libs']
  NOT AVAILABLE

openblas_info:
  libraries openblas not found in ['C:\\ProgramData\\Miniconda3\\lib', 'C:\\', 'C:\\ProgramData\\Miniconda3\\libs']


Running from numpy source directory.
Traceback (most recent call last):
  File "setup.py", line 488, in <module>
    setup_package()
  File "setup.py", line 480, in setup_package
    setup(**metadata)
  File "C:\Users\feelus\Repos\__foreign\numpy\numpy\distutils\core.py", line 135, in setup
    config = configuration()
  File "setup.py", line 162, in configuration
    config.add_subpackage('numpy')
  File "C:\Users\feelus\Repos\__foreign\numpy\numpy\distutils\misc_util.py", line 1020, in add_subpackage
    caller_level = 2)
  File "C:\Users\feelus\Repos\__foreign\numpy\numpy\distutils\misc_util.py", line 989, in get_subpackage
    caller_level = caller_level + 1)
  File "C:\Users\feelus\Repos\__foreign\numpy\numpy\distutils\misc_util.py", line 926, in _get_configuration_from_setup_py
    config = setup_module.configuration(*args)
  File "numpy\setup.py", line 8, in configuration
    config.add_subpackage('core')
  File "C:\Users\feelus\Repos\__foreign\numpy\numpy\distutils\misc_util.py

In [1]:
import sys

print(sys.executable)

C:\ProgramData\Miniconda3\python.exe


In [4]:
sys.version_info[:2]

(3, 6)

In [8]:
(1,2)<(1,3)

True